# 인스타그램 검색어에 대한 해시태그 크롤링

코드 참고: https://github.com/Yeowoolee/Instagram-crawler

In [1]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

In [2]:
search = 'bubble_tea' # 검색어

In [3]:
url = 'https://www.instagram.com/explore/tags/'+str(search)+'/' # 검색어에 대한 결과

In [4]:
driver = webdriver.Chrome('./chromedriver.exe') 

In [5]:
driver.get(url)
sleep(5) #로딩 시간

In [6]:
SCROLL_PAUSE_TIME = 1.5 #로딩 시간

In [7]:
linklist = []

In [8]:
while True:
    pageString = driver.page_source
    bsObj = BeautifulSoup(pageString, "lxml")

    for link1 in bsObj.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):

            title = link1.select('a')[0] 
            link = title.attrs['href']
            linklist.append(link) 
            title = link1.select('a')[1] 
            link = title.attrs['href']
            linklist.append(link) 
            title = link1.select('a')[2] 
            link = title.attrs['href']
            linklist.append(link)

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        else:
            last_height = new_height
            continue

In [9]:
len(linklist)

8262

In [10]:
# 너무 길어서 시간이 오래 걸려서 일단 10개만 합니다
ten = linklist[0:10]

In [11]:
def id_and_hashtag_to_csv(linklist, search):
    df = [['insta_account', 'hashtag']]
    for l in linklist:
        rows = []
        
        req = Request('https://www.instagram.com/p'+l,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage,"lxml",from_encoding='utf-8')
        soup1 = soup.find("meta",attrs={"property":"og:description"})
        
        insta_account = soup1['content']
        insta_account = insta_account[insta_account.find("@")+1:insta_account.find(")")]
        insta_account = insta_account[:20]
        if insta_account == '':
            continue
        
        for hashtag in soup.find_all("meta",attrs={"property":"instapp:hashtags"}):
            hashtag = hashtag['content']
            hashtag = hashtag[:10]
            
            rows.append([insta_account, hashtag])
        
        df.extend(rows)
    
    data = pd.DataFrame(df)
    data.to_csv('{}.csv'.format(search), mode='w',encoding='utf-8',header=None)

In [12]:
id_and_hashtag_to_csv(ten, search)

In [13]:
import json

In [14]:
def id_and_hashtag_to_json(linklist, search):
    dt = dict()
    
    for l in linklist:
        req = Request('https://www.instagram.com/p'+l,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage,"lxml",from_encoding='utf-8')
        soup1 = soup.find("meta",attrs={"property":"og:description"})
        
        insta_account = soup1['content']
        insta_account = insta_account[insta_account.find("@")+1:insta_account.find(")")]
        insta_account = insta_account[:20]
        if insta_account == '':
            continue
        
        if insta_account in dt.keys():
            pass
        else:
            dt[insta_account] = []
            
        for hashtag in soup.find_all("meta",attrs={"property":"instapp:hashtags"}):
            hashtag = hashtag['content']
            hashtag = hashtag[:10]
            dt[insta_account].append(hashtag)
    
    with open('{}.json'.format(search),'w') as fp:
        json.dump(dt, fp, indent=4)

In [15]:
id_and_hashtag_to_json(ten, search)

In [17]:
# 브라우저 끄기
driver.quit()